# Project Proposal: Building a Classifier to Predict Diabetes

# 1. Introduction

# 2. Preliminary Analysis

Our GitHub repository can be found at: https://github.com/hesoru/DSCI_100_Diabetes_Prediction

### Dataset Source

Our dataset was obtained from Kaggle at the following URL: 

https://www.kaggle.com/datasets/mathchi/diabetes-data-set?fbclid=IwAR1DMzdJFDxoEqLDIZNTi3j7YJXTx_7BJwCl7sbn8syQKbQCnHfMtlsKH1E

This dataset was uploaded by user Mehmet Akturk and was sourced from the National Institute of Diabetes and Digestive and Kidney Diseases (donated by RMI Group Leader Vincent Sigillito).

In [24]:
library("tidyverse")
library("tidymodels")

ERROR: Error: package or namespace load failed for ‘tidymodels’ in loadNamespace(i, c(lib.loc, .libPaths()), versionCheck = vI[[i]]):
 namespace ‘rlang’ 1.1.0 is already loaded, but >= 1.1.1 is required


## Tidying Data

In [13]:
diabetes_dataset <- read_csv("Dataset/diabetes.csv")
diabetes_dataset
# dataset appears to have no N/A values, will check with drop_na

diabetes_dataset_no_NA <- drop_na(diabetes_dataset)
dim(diabetes_dataset_no_NA)
dim(diabetes_dataset)
# both have the same dimensions, so data is already clean

Rows: 768 Columns: 9
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (9): Pregnancies, Glucose, BloodPressure, SkinThickness, Insulin, BMI, D...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
6,148,72,35,0,33.6,0.627,50,1
1,85,66,29,0,26.6,0.351,31,0
8,183,64,0,0,23.3,0.672,32,1
1,89,66,23,94,28.1,0.167,21,0
0,137,40,35,168,43.1,2.288,33,1
5,116,74,0,0,25.6,0.201,30,0
3,78,50,32,88,31.0,0.248,26,1
10,115,0,0,0,35.3,0.134,29,0
2,197,70,45,543,30.5,0.158,53,1


[1] 768   9

[1] 768   9

## Split Clinical Data into Training and Testing Sets

In [16]:
diabetes_dataset_split <- initial_split(data = diabetes_dataset, prop = 0.75, strata = Outcome)
diabetes_dataset_training <- training(clinical_data_tidy_split)
diabetes_dataset_testing <- testing(clinical_data_tidy_split)

ERROR: Error in initial_split(data = diabetes_dataset, prop = 0.75, strata = Outcome): could not find function "initial_split"
